In [1]:
import importlib
import warnings
from logging import getLogger
from types import ModuleType
from process_bigraph import ProcessTypes
from bigraph_schema.protocols import function_module
from unum import Unum
from importlib import import_module
from typing import *

from wholecell.utils import units
from ecoli.shared import ecoli_core as ec
from ecoli.shared.types.register import test_register_type

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [2]:
test_register_type(ec)

In [3]:
ec.types().get('unum')

{'_apply': 'apply_unum',
 '_check': 'check_unum',
 '_divide': 'divide_unum',
 '_serialize': 'serialize_unum',
 '_deserialize': 'deserialize_unum',
 '_type': 'unum',
 '_default': 1.0 [s/mol]}